In [1]:
# Preamble

import os
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline  

#os.chdir("/Users/macuser/Downloads/[ML]") 

/Users/macuser/anaconda/lib/python3.4/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [ ]:
# Read Data

df = pd.read_csv(dataname)
#df = pd.read_excel(dataname, sheetname=page)

print ('Data has {} rows and {} columns'.format(df.shape))

In [ ]:
# Look at columns

df.columns.values

## Optional: replace column names

In [ ]:
# Duplicates and Missing Values

print('Number of duplicated observations:', df.duplicated().sum())
print('Number of missing values:', df['ref_domain_name'].isnull().sum())


## Optional: trim data based on duplicates and missing values
#df.drop_duplicates(inplace=True)
#df = df.dropna(['Column1'])

In [ ]:
# Describe Variables in Batch:

#describe non-numeric values
if (df.select_dtypes(include=['object']).shape[1]>0):
    des_obj = np.round(df.select_dtypes(include=['object']).describe(), 2).T
    print ('Object columns:',des_obj)
    
#Describe the rest
des_num = np.round(df.select_dtypes(exclude=['object']).describe(), 2).T
print ('Numeric columns:',des_num)

In [ ]:
# Snapshots of all kinds:

## Sort:
df.sort(['A', 'B'], ascending=[1, 0])

## Count values:
df['A'].value_counts()
# df['A'].value_counts(sort=False)

## Row Slicing:
df[101:105]
## Column slicing by number
df.iloc[:,2:4]
## Column slicing by label
df.loc[:,['ref_domain_name','domain_name']]

In [ ]:
# Numerical value manipulation

np.round(df['A'], 2)
df[log_A] = df[A].apply(lambda x: np.log(x+1))
df['A'].quantile([0.05, 0.1, 0.5, 0.9, 0.95])

bin = np.linspace(20,110,19)

In [ ]:
# Graphic representation

bin = np.linspace(0,12,13)
plt.hist(df['A'].dropna(),bins=bin)
plt.hist(df['A'].dropna(),bins=bin,log=True)

In [ ]:
# Relationship across variables

## Correlation:
df.iloc[:,[3,9,7]].corr()

## Summary statistics by group:
grouped = df.groupby(['A', 'B'])
df.groupby('A').groups
len(grouped)

grouped.get_group('amazon.com')['basket_tot']
grouped.get_group('amazon.com')['basket_tot'].sum()
grouped.get_group('amazon.com')['basket_tot'].mean()
grouped.get_group('amazon.com')['basket_tot'].std()
grouped.get_group('amazon.com')['basket_tot'].boxplot()



## apply function to each group (the function inside 'apply' can be user-defined function as well)
grouped['A'].apply(lambda x: x.describe())


## Cross tabulation (aggfunc example: np.sum, np.std, np.mean,...):
table1 = pd.crosstab(df.A, df.B, values=df.C, aggfunc=np.sum)

print(table1.to_string(na_rep=''))

In [ ]:
# Data Transformation

## get dummies and join it to the original df
dummies = pd.get_dummies(df['machine_id'], prefix='id')
df.join(dummies)

## standarlization/scaling

## replace missing value

## transform dtype (to, e.g. 'float', 'int', 'str', 'category')
df["col"].astype("category")